In [0]:
!pip install numpy==1.15.4


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
urlLink = 'https://drive.google.com/open?id=1F3huUVDkqw7RuOHOFVyHyGAqkzc8zbSA'
fluff, ide = urlLink.split('=')

 
url= 'https://drive.google.com/open?id=10Xr4onhDh-cl1-G7v_Pu0bCv8AZ8v6uJ'
alpha, bravo = url.split('=')


In [0]:
import pandas as pd
import os
import fnmatch
import cv2
import numpy as np
import string
import time

from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dropout, Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional,Dropout
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [0]:
downloaded = drive.CreateFile({'id':ide})

downloaded.GetContentFile('train_data_processed.h5')
downloaded_1.GetContentFile('test_data_processed.h5')

df3 = pd.read_hdf('train_data_processed.h5','table','r') 
df4 = pd.read_hdf('test_data_processed.h5','table','r')


In [0]:
from sklearn.utils import shuffle
df3 = shuffle(df3)

In [0]:
df4['num_digits'].unique()

array([1, 2, 3, 4, 5])

In [0]:
import tensorflow as tf

#ignore warnings in the output
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
from tensorflow.python.client import device_lib

# Check all available devices if GPU is available
print(device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Training and validation

In [0]:
# lists for training dataset
training_img = []
training_txt = []
train_input_length = []
train_label_length = []
orig_txt = []
 
#lists for validation dataset
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []
 
max_label_len = 0
 
i = 1 

In [0]:
# for testing purpose

for i in range(df4['cut_img'].values.shape[0]):
        # read input image and convert into gray scale image
        img = df4['cut_img'][i] 
        img[:] = np.max(img,axis=-1,keepdims=1)/2+np.min(img,axis=-1,keepdims=1)/2      
        num_digits=df4['num_digits']
        h, w, d=img.shape
        img = img.reshape(h,w*d)
        #img[:] = np.max(img,axis=-1,keepdims=1)/2+np.min(img,axis=-1,keepdims=1)/2 
        
        h, w=img.shape

        if w > 128 or h > 32:
            img =cv2.resize(img, dsize=(128,32), interpolation=cv2.INTER_AREA)
        else:
          if h < 32:
            add_zeros = np.ones((32-h, w))*255
            img = np.concatenate((img, add_zeros))
            
          if w < 128:
            add_zeros = np.ones((32, 128-w))*255
            img = np.concatenate((img, add_zeros), axis=1)
        
        img = np.expand_dims(img , axis = 2)
        
        # Normalize each image
        img = img/255
        #print(img.shape)
        text=df4['labels'][i]
        output =""
        for key,value in enumerate(text):
          if value=='.':
            output=output+text[key-1]

        encode_output=list(output)
        
        if len(num_digits) > max_label_len:
            max_label_len = len(output)

        valid_orig_txt.append(output)   
        valid_label_length.append(len(output))
        valid_input_length.append(31)
        valid_img.append(img)
        valid_txt.append(encode_output)
        
        # break the loop if total data is 33402
        if i == 13000:
            break
        i+=1


In [0]:
char_list = string.digits

valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

valid_img = np.array(valid_img)
valid_input_length = np.array(valid_input_length)
valid_label_length = np.array(valid_label_length)

In [0]:
# for training and validation purpose

for i in range(df3['cut_img'].values.shape[0]):
        # read input image and convert into gray scale image
        img = df3['cut_img'][i] 
        img[:] = np.max(img,axis=-1,keepdims=1)/2+np.min(img,axis=-1,keepdims=1)/2      
        num_digits=df3['num_digits']
        h, w, d=img.shape
        img = img.reshape(h,w*d)
        
        h, w=img.shape

        if w > 128 or h > 32:
            img =cv2.resize(img, dsize=(128,32), interpolation=cv2.INTER_AREA)
        else:
          if h < 32:
            add_zeros = np.ones((32-h, w))*255
            img = np.concatenate((img, add_zeros))
            
          if w < 128:
            add_zeros = np.ones((32, 128-w))*255
            img = np.concatenate((img, add_zeros), axis=1)
        
        img = np.expand_dims(img , axis = 2)
        
        # Normalize each image
        img = img/255
        #print(img.shape)
        text=df3['labels'][i]
        output =""
        for key,value in enumerate(text):
          if value=='.':
            output=output+text[key-1]

        encode_output=list(output)
        
        if len(num_digits) > max_label_len:
            max_label_len = len(output)

        if i%10 == 0:     
            valid_orig_txt.append(output)   
            valid_label_length.append(len(output))
            valid_input_length.append(31)
            valid_img.append(img)
            valid_txt.append(encode_output)
        else:
            orig_txt.append(output)   
            train_label_length.append(len(output))
            train_input_length.append(31)
            training_img.append(img)
            training_txt.append(encode_output) 

        # break the loop if total data is 33402
        if i == 33000:
            break
        i+=1


In [0]:
# pad each output label to maximum text length
char_list = string.digits

train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

In [0]:
# input with shape of height=32 and width=128 
inputs = Input(shape=(32,128,1))
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
batch_norm_1 = BatchNormalization()(conv_1)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(batch_norm_1)
dropout_1 = Dropout(0.4)(pool_1)

pool_2 = MaxPool2D(pool_size=(2, 1))(dropout_1)

conv_3 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_2)
batch_norm_3 = BatchNormalization()(conv_3)
pool_3 = MaxPool2D(pool_size=(2, 2), strides=2)(batch_norm_3)

pool_4 = MaxPool2D(pool_size=(2, 1))(pool_3) #add conv2d here maybe

conv_5 = Conv2D(128, (2,2), activation = 'relu')(pool_4)
batch_norm_5 = BatchNormalization()(conv_5)

squeezed = Lambda(lambda x: K.squeeze(x, 1))(batch_norm_5)
 
# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(32, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(32, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

# model to be used at test time
actual_model = Model(inputs, outputs)

In [0]:
actual_model.summary()

In [0]:
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
 
 
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

#model to be used at training time
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

In [0]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adadelta',metrics = ['accuracy'])
 
filepath="best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

In [0]:
# for training

training_img = np.array(training_img)
train_input_length = np.array(train_input_length)
train_label_length = np.array(train_label_length)

valid_img = np.array(valid_img)
valid_input_length = np.array(valid_input_length)
valid_label_length = np.array(valid_label_length)

In [0]:
batch_size = 256
epochs = 200
history= model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length], y=np.zeros(len(training_img)), batch_size=batch_size, epochs = epochs, validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], [np.zeros(len(valid_img))]), verbose = 1, callbacks = callbacks_list)

In [0]:
from keras.models import load_model

act_model.load_weights('best_model.hdf5')

In [0]:
# for prediction

training_img = np.array(training_img)
train_input_length = np.array(train_input_length)
train_label_length = np.array(train_label_length)

valid_img = np.array(valid_img)
valid_input_length = np.array(valid_input_length)
valid_label_length = np.array(valid_label_length)

In [0]:
# predict outputs on validation images
prediction = act_model.predict(valid_img[:13000])
 
# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
 
total = 0
count = 0
var2 = ""
i = 0
for x in out:
    var2 = ""
    print("original_text =  ", valid_orig_txt[i])
    print("predicted text = ", end = '')
    for p in x:  
        if int(p) != -1:
            var2 = var2+char_list[int(p)]
            print(char_list[int(p)], end = '')
  
    print('\n')
    if var2 == valid_orig_txt[i]:
      total=total+1
      count=count+1
    else:
      total=total+1
    i+=1

original_text =   5
predicted text = 5

original_text =   16
predicted text = 16

original_text =   61
predicted text = 61

original_text =   27
predicted text = 271

original_text =   143
predicted text = 143

original_text =   131
predicted text = 131

original_text =   60
predicted text = 60

original_text =   44
predicted text = 44

original_text =   110
predicted text = 110

original_text =   1
predicted text = 1

original_text =   80
predicted text = 99

original_text =   24
predicted text = 24

original_text =   77
predicted text = 77

original_text =   172
predicted text = 173

original_text =   119
predicted text = 119

original_text =   717
predicted text = 717

original_text =   172
predicted text = 72

original_text =   82
predicted text = 82

original_text =   22
predicted text = 22

original_text =   67
predicted text = 37

original_text =   16
predicted text = 16

original_text =   69
predicted text = 69

original_text =   64
predicted text = 44

original_text =   20
pre

In [0]:
answer=(count/total)*100
print("accuracy is {}%".format(answer))

print(count)
print(total)

accuracy is 79.33076923076922%
10313
13000


In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()